# 1. Objetivo
O objetivo deste notebook é reproduzir a metodologia do cálculo de consumo aparente, apresentado no artigo [Indicadores de Consumo Aparente de Bens Industriais: metodologia e resultados](http://www.ipeadata.gov.br/doc/td_2101.pdf).

**Autor:** Amilton Lobo Mendes Júnior
amilton.mendes@gmail.com

# 2. Preparação do ambiente


## 2.1 Instalação de bibliotecas utilizadas

In [75]:
!pip install pyIpeaData

##2.2. Import das bibliotecas utilizadas neste notebook





In [35]:
import pandas as pd
import requests
import json
from pandas import json_normalize
from datetime import datetime
import pyIpeaData as ip

## 2.3 URL das tabelas de recursos e usos de 2021, armazenada no github.

In [36]:
url_recursos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab1_2019.xls'
url_usos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab2_2019.xls'


## 2.4 Dicionários com informações sobre os dados por setor

In [37]:
#Criar um mapa que faça a correlação entre um código de cnae e códigos para baixar a série de importação, exportação e a url para download de dados do sidra

mapa_cnae = {
    "2100": {  # CNAE 2100 - Fabricação de produtos farmacêuticos
        "importacao": "FUNCEX12_MQFARM2N12",
        "exportacao": "FUNCEX12_XQFARM2N12",
        "sidra_url": "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129330/d/v12606%205"
    },
    "29" : {
        "importacao": "FUNCEX12_MQVEIC2N12",
        "exportacao": "FUNCEX12_XQVEIC2N12",
        "sidra_url": "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129338/d/v12606%205"
    }
    # adicione mais CNAEs aqui...
}



## 2.5 Funções e classes utilizadas no notebook

In [48]:
#Carrega dados oferta
def carrega_dados_oferta():
  oferta = pd.read_excel(url_recursos, sheet_name='oferta',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  #Renomear as duas primeiras colunas que não foram carregadas por estarem mescladas
  oferta= oferta.rename(columns={oferta.columns[0] : 'Código do produto', oferta.columns[1] : 'Descrição do produto'})
  #Remover a primeira linha (vazia)
  oferta = oferta.iloc[1:]
  return oferta
def carrega_dados_producao():
  producao = pd.read_excel(url_recursos, sheet_name='producao',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  producao = producao.rename(columns={producao.columns[0] : 'Código do produto', producao.columns[1] : 'Descrição do produto'})
  producao = producao.iloc[1:]
  return producao

def carrega_dados_importacao():
  importacao = pd.read_excel(url_recursos, sheet_name='importacao',skiprows=3,skipfooter=5,dtype={'Unnamed: 0' : str})
  importacao = importacao.rename(columns={importacao.columns[0] : 'Código do produto', importacao.columns[1] : 'Descrição do produto'})
  importacao = importacao.iloc[1:]
  return importacao

def carrega_dados_exportacao():
  exportacao = pd.read_excel(url_usos, sheet_name='demanda',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  exportacao = exportacao.rename(columns={exportacao.columns[0] : 'Código do produto', exportacao.columns[1] : 'Descrição do produto'})
  exportacao = exportacao.iloc[1:]
  return exportacao

class ConsumoAparente():
  def __init__(self,oferta,producao,importacao,exportacao):
    self.oferta = oferta
    self.producao = producao
    self.importacao = importacao
    self.exportacao = exportacao
  def carrega_dados_producao_mensal(self,url):
    r = requests.get(url)
    #Apresentar data de carregamento
    a = json.loads(r.text)
    res = json_normalize(a)

    producao = pd.DataFrame(res)
    nomes_colunas = producao.iloc[0].values
    producao.columns = nomes_colunas
    producao = producao.iloc[1:]
    producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
    producao['Ano'] = producao['Mês (Código)'].str[:4]
    producao['Mes'] = producao['Mês (Código)'].str[4:]

    producao['Ano'] = producao['Ano'].astype(int)
    producao['Mes'] = producao['Mes'].astype(int)

    producao = producao[['Ano','Mes','Valor']]
    producao = producao.rename(columns={'Valor' : 'producao'})

    producao['producao'] = producao['producao'].astype(float)

    return producao.copy()
  def carrega_dados_quantum_exp_imp(self,url,movimento):
    ##Carrega dados de quantum de exportação e importação
    dados_movimento = ip.get_serie(url)

    dados_movimento['VALDATA'] = pd.to_datetime(dados_movimento['VALDATA'],utc=True)

    dados_movimento['Ano'] = dados_movimento['VALDATA'].dt.year
    dados_movimento['Mes'] = dados_movimento['VALDATA'].dt.month

    dados_movimento = dados_movimento[['Ano','Mes','VALVALOR']]

    dados_movimento = dados_movimento.rename(columns={'VALVALOR' : movimento})
    dados_movimento[movimento] = dados_movimento[movimento].astype(float)

    return dados_movimento

  def calcula_consumo_aparente(self,cnae,verbose=0):
    preco_consumidor,preco_basico = self.oferta[self.oferta['Código do produto'].str.startswith(cnae)][['Oferta total\na preço\nde consumidor','Oferta total\n a preço básico ']].sum().values
    fator_ajuste = preco_basico/preco_consumidor
    valor_producao = self.producao[self.producao['Código do produto'].str.startswith(cnae)][['Total\ndo produto']].sum().values[0]
    valor_exportacao = self.exportacao[self.exportacao['Código do produto'].str.startswith(cnae)][['Exportação\nde bens e\nserviços (1)']].sum().values[0]
    valor_exportacao = valor_exportacao*fator_ajuste
    valor_importacao = self.importacao[self.importacao['Código do produto'].str.startswith(cnae)][['Importação de bens\ne serviços (1)']].sum().values[0]
    #Calcular pesos de acordo com fórmula constante do arquivo
    total = valor_importacao - valor_exportacao + valor_producao
    peso_importacao = valor_importacao/total
    peso_exportacao = valor_exportacao/total
    peso_producao = valor_producao/total

    #Prepara dados de quantum de exportação e importação para a junção com dados de produção e cálculo de consumo aparente
    if verbose == 1:
      print(f'Preço de consumidor: {preco_consumidor}')
      print(f'Preço básico: {preco_basico}')
      print(f'Fator de ajuste: {fator_ajuste}')
      print(f'Peso da importação: {peso_importacao}')
      print(f'Peso da exportação: {peso_exportacao}')
      print(f'Peso da produção: {peso_producao}')
      print('Carregando dados de produção da PIM-PF (Sidra)')
    producao = self.carrega_dados_producao_mensal(mapa_cnae[cnae]['sidra_url'])
    #Converter esse índice para a base de 2012
    media_indice_2012 = producao[producao['Ano'] == 2012]['producao'].mean()
    producao['producao'] = producao['producao']*100/media_indice_2012

    if verbose == 1:
      print('Carregando dados de exportação e importação da Quantum ( Funcex - Ipeadata)')
    dados_exportacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[cnae]['exportacao'],'exportacao')
    #Converter esse índice para a base de 2012
    media_indice_2012 = dados_exportacao[dados_exportacao['Ano'] == 2012]['exportacao'].mean()
    dados_exportacao['exportacao'] = dados_exportacao['exportacao']*100/media_indice_2012
    dados_importacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[cnae]['importacao'],'importacao')
    #Converter esse índice para a base de 2012
    media_indice_2012 = dados_importacao[dados_importacao['Ano'] == 2012]['importacao'].mean()
    dados_importacao['importacao'] = dados_importacao['importacao']*100/media_indice_2012

    #Juntar os dados de produção, importação e exportação
    producao = producao.merge(dados_exportacao,how='inner',on=['Ano','Mes']).merge(dados_importacao,how='inner',on=['Ano','Mes'])



    producao['producao'] = producao['producao'].astype(float)
    producao['importacao'] = producao['importacao'].astype(float)
    producao['exportacao'] = producao['exportacao'].astype(float)

    producao['producao_base_movel'] = producao['producao'].pct_change()*100
    producao['exportacao_base_movel'] = producao['exportacao'].pct_change()*100
    producao['importacao_base_movel'] = producao['importacao'].pct_change()*100
    
    producao['consumo_aparente'] = producao['producao']*peso_producao - producao['exportacao']*peso_exportacao + producao['importacao']*peso_importacao
    producao['consumo_aparente_base_movel'] = producao['producao_base_movel']*peso_producao - producao['exportacao_base_movel']*peso_exportacao + producao['importacao_base_movel']*peso_importacao

    producao['consumo_aparente_12_meses_anterior'] = producao['consumo_aparente'].shift(12)

    producao['variacao_consumo_aparente'] = ((producao['consumo_aparente']/producao['consumo_aparente_12_meses_anterior'])-1)*100

    producao['variacao_consumo_aparente_base_movel'] = producao['consumo_aparente_base_movel'].pct_change(12)*100

    return producao






In [49]:
consumoAparente = ConsumoAparente(carrega_dados_oferta(),carrega_dados_producao(),carrega_dados_importacao(),carrega_dados_exportacao())

In [50]:
consumoAparente.calcula_consumo_aparente('29',1).tail(13)

Preço de consumidor: 509561.0
Preço básico: 370892.0
Fator de ajuste: 0.7278657511073258
Peso da importação: 0.19384516594722254
Peso da exportação: 0.10051617640706666
Peso da produção: 0.9066710104598441
Carregando dados de produção da PIM-PF (Sidra)
Carregando dados de exportação e importação da Quantum ( Funcex - Ipeadata)


,Ano,Mes,producao,exportacao,importacao,producao_base_movel,exportacao_base_movel,importacao_base_movel,consumo_aparente,consumo_aparente_base_movel,consumo_aparente_12_meses_anterior,variacao_consumo_aparente,variacao_consumo_aparente_base_movel
264,2024,1,57.410210,59.662383,88.390620,0.536868,-20.469997,-3.630780,63.189233,1.840519,58.217656,8.539637,-121.590061
265,2024,2,68.076054,76.559887,70.306578,18.578306,28.321873,-20.459231,67.655668,10.031682,60.636295,11.576190,97.879237
266,2024,3,69.854460,82.525016,105.762234,2.612381,7.791455,50.430070,75.541413,11.361028,74.723624,1.094418,-50.188965
267,2024,4,80.633663,90.197636,108.131468,15.430944,9.297326,2.240150,85.002645,13.490501,61.137847,39.034412,-175.502525
268,2024,5,70.236014,80.917314,119.265206,-12.894924,-10.288875,10.296484,78.666442,-8.661331,74.517283,5.568049,-140.033364
269,2024,6,71.950928,68.975968,178.819111,2.441645,-14.757468,49.934014,92.965840,13.376600,66.651544,39.480401,-227.468706
270,2024,7,81.090779,88.490146,106.101880,12.702896,28.291271,-40.665246,85.195304,0.790856,63.807172,33.519950,-118.404835
271,2024,8,84.552071,87.159634,117.318459,4.268415,-1.503571,10.571518,90.641574,6.070419,77.186858,17.431357,-69.228025
272,2024,9,82.813402,103.358619,104.652174,-2.056329,18.585422,-10.796498,84.981615,-5.825398,71.876530,18.232774,-17.967107
273,2024,10,89.551160,107.826923,129.901903,8.136073,4.323107,24.127286,95.535947,11.619157,71.787563,33.081474,-2481.556094


In [25]:
consumoAparente.calcula_consumo_aparente('2100',0)

,Ano,Mes,producao,exportacao,importacao,consumo_aparente,consumo_aparente_12_meses_anterior,variacao_consumo_aparente_arredondado
0,2002,1,55.42792,19.73,32.93,48.844609,NaN,NaN
1,2002,2,76.43190,18.53,29.31,62.232952,NaN,NaN
2,2002,3,86.01877,21.86,40.93,72.645370,NaN,NaN
3,2002,4,91.12036,29.63,42.62,76.552126,NaN,NaN
4,2002,5,78.10670,23.99,33.14,64.529515,NaN,NaN
...,...,...,...,...,...,...,...,...
272,2024,9,102.27364,107.38,145.04,116.313949,111.759507,4.075217
273,2024,10,109.16742,104.73,167.95,128.744501,124.347666,3.535920
274,2024,11,101.50792,116.98,140.18,113.940092,109.335135,4.211781
275,2024,12,75.63582,101.94,141.96,96.963116,80.056770,21.117947


In [28]:
producao = consumoAparente.carrega_dados_producao_mensal(mapa_cnae['2100']['sidra_url'])


In [31]:
producao.tail(14)

,Ano,Mes,producao
265,2024,1,72.81993
266,2024,2,82.38201
267,2024,3,92.89157
268,2024,4,100.29527
269,2024,5,103.71688
270,2024,6,107.13063
271,2024,7,102.94700
272,2024,8,110.04800
273,2024,9,102.27364
274,2024,10,109.16742


In [32]:
variacao_producao_12m = producao['producao'].pct_change(periods=12)*100
producao['variacao_producao_12m'] = variacao_producao_12m
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].round(2)
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].astype(float)
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].astype(str).str.replace('nan','-')
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].str.replace('inf','-')
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].str.replace('-','-')



In [34]:
producao.tail(14)

,Ano,Mes,producao,variacao_producao_12m
265,2024,1,72.81993,-17.24
266,2024,2,82.38201,-18.73
267,2024,3,92.89157,-16.41
268,2024,4,100.29527,-6.55
269,2024,5,103.71688,1.98
270,2024,6,107.13063,16.26
271,2024,7,102.94700,6.06
272,2024,8,110.04800,-6.72
273,2024,9,102.27364,3.37
274,2024,10,109.16742,4.68
